# Creating the trainable dataset and splitting it to validation subset

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import json

In [ ]:
file = open('../input/pokemonnames/pokemon_names.json', 'r')
names = file.read()
NAMES = json.loads(names)

In [ ]:
# Specifying the size of image that is being processed (resizing if not) and setting the batch size for training
img_width = 96
img_height = 96
batch_size = 5

In [ ]:
# Training Subset
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    '../input/pokemonclassification/PokemonData/',
    label_mode='categorical',
    batch_size = batch_size,
    image_size = (img_height, img_width),
    seed = 123,
    validation_split = 0.1,
    subset = 'training',
)

# Validation Subset
ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    '../input/pokemonclassification/PokemonData/',
    label_mode='categorical',
    batch_size = batch_size,
    image_size = (img_height, img_width),
    seed = 123,
    validation_split = 0.1,
    subset = 'validation',
)

In [ ]:
# We use this to avoid using I/O and yeild data from disk
AUTOTUNE = tf.data.AUTOTUNE
ds_train = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
ds_validation = ds_validation.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# As the images in the dataset are less we are augmenting the data to form more images to train on
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
  ]
)

# Model making starts from here

In [ ]:
model = keras.Sequential([
    data_augmentation,
    layers.Rescaling(1./255, input_shape=(img_width, img_height, 3)),

    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.BatchNormalization(axis = -1),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, 3, padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(axis = -1),
    layers.Conv2D(64, 3, padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(axis = -1),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, 3, padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(axis = -1),
    layers.Conv2D(128, 3, padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(axis = -1),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(256, 3, padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(axis = -1),
    layers.Conv2D(256, 3, padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(axis = -1),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Flatten(),
#     layers.Dense(4097, activation='relu'), 
#     layers.BatchNormalization(),
#     layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(len(NAMES), activation='softmax')
])

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    # loss = [keras.losses.SparseCategoricalCrossentropy(from_logits=True),],
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
)

summary = model.fit(ds_train, validation_data = ds_validation, epochs=150, verbose=0)


# Saving the model for future use

In [ ]:
model.save('./model/PokeModels')

# Image Detection using the above model

In [ ]:
import cv2
import numpy as np

In [ ]:
def prepare(filepath):
    IMG_SIZE = 96
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [ ]:
prediction = model.predict([(prepare('../input/test-objects/pika.jpg'))])
score = tf.nn.softmax(prediction[0])
pokemon = NAMES[np.argmax(score)]
print("This image most likely belongs to {} with a {:.2f} confidence.".format(pokemon, 100*np.max(score)))

# Installing Python Text To Speech module (Under development)

In [ ]:
!pip install pyttsx3

# Pokemon Information

In [ ]:
import pyttsx3
engine = pyttsx3.init()

In [ ]:
def Say(message):
    engine.say(message)
    engine.runAndWait()